In [1]:
# Libraries 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
import math
import string

In [2]:
# Import data
chopped_df = pd.read_csv('chopped.csv')

In [3]:
#printing all summary of the chopped data
#this will give the dimensions of data set : (rows, columns)
print(chopped_df.shape)
#columns and data types
print(chopped_df.info())
#basic stats of columns
print(chopped_df.describe())

(569, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 20 columns):
season              569 non-null int64
season_episode      569 non-null int64
series_episode      569 non-null int64
episode_name        569 non-null object
episode_notes       456 non-null object
air_date            569 non-null object
judge1              568 non-null object
judge2              568 non-null object
judge3              568 non-null object
appetizer           569 non-null object
entree              569 non-null object
dessert             568 non-null object
contestant1         568 non-null object
contestant1_info    556 non-null object
contestant2         569 non-null object
contestant2_info    555 non-null object
contestant3         569 non-null object
contestant3_info    556 non-null object
contestant4         569 non-null object
contestant4_info    555 non-null object
dtypes: int64(3), object(17)
memory usage: 89.0+ KB
None
           season  season_episo

In [4]:
# Determining the winner of each episode
chopped_df['winner'] = chopped_df['contestant4']

In [5]:
# Determining amount of injuries
chopped_df['injured'] = chopped_df['episode_notes'].str.contains("cut himself|cut herself|burnt|burns")

In [6]:
chopped_df

,season,season_episode,series_episode,episode_name,episode_notes,air_date,judge1,judge2,judge3,appetizer,...,contestant1,contestant1_info,contestant2,contestant2_info,contestant3,contestant3_info,contestant4,contestant4_info,winner,injured
0,1,1,1,"""Octopus, Duck, Animal Crackers""",This is the first episode with only three offi...,"January 13, 2009",Marc Murphy,Alex Guarnaschelli,Aarón Sánchez,"baby octopus, bok choy, oyster sauce, smoked ...",...,Summer Kriegshauser,Private Chef and Nutrition Coach New York NY,Perry Pollaci,Private Chef and Sous chef Bar Blanc New Yo...,Katie Rosenhouse,Pastry Chef Olana Restaurant New York NY,Sandy Davis,Catering Chef Showstoppers Catering at Union...,Sandy Davis,False
1,1,2,2,"""Tofu, Blueberries, Oysters""",This is the first of a few episodes with five ...,"January 20, 2009",Aarón Sánchez,Alex Guarnaschelli,Marc Murphy,"firm tofu, tomato paste, prosciutto",...,Raymond Jackson,Private Caterer and Culinary Instructor West...,Klaus Kronsteiner,Chef de cuisine Liberty National Golf Course...,Christopher Jackson,Executive Chef and Owner Ted and Honey Broo...,Pippa Calland,Owner and Chef Chef for Hire LLC Newville PA,Pippa Calland,False
2,1,3,3,"""Avocado, Tahini, Bran Flakes""",NaN,"January 27, 2009",Aarón Sánchez,Alex Guarnaschelli,Marc Murphy,"lump crab meat, dried shiitake mushrooms, pin...",...,Margaritte Malfy,Executive Chef and Co-owner La Palapa New Y...,Rachelle Rodwell,Chef de cuisine SoHo Grand Hotel New York NY,Chris Burke,Private Chef New York NY,Andre Marrero,Chef tournant L’Atelier de Joël Robuchon Ne...,Andre Marrero,NaN
3,1,4,4,"""Banana, Collard Greens, Grits""","In the appetizer round, Chef Chuboda refused t...","February 3, 2009",Scott Conant,Amanda Freitag,Geoffrey Zakarian,"ground beef, wonton wrappers, cream of mushro...",...,Sean Chudoba,Executive Chef Ayza Wine Bar New York NY,Kyle Shadix,Chef Registered Dietician and Culinary Consu...,Luis Gonzales,Executive Chef Knickerbocker Bar & Grill Ne...,Einat Admony,Chef and Owner Taïm New York NY,Einat Admony,False
4,1,5,5,"""Yucca, Watermelon, Tortillas""",NaN,"February 10, 2009",Geoffrey Zakarian,Alex Guarnaschelli,Marc Murphy,"watermelon, canned sardines, pepper jack chee...",...,John Keller,Personal Chef New York NY,Andrea Bergquist,Executive Chef New York NY,Ed Witt,Executive Chef / Wine Director Bloomingdale ...,Josh Emett,Chef de cuisine Gordon Ramsay at The London ...,Josh Emett,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,45,9,563,"""Terrine Cuisine""",Chef Jose cut himself in the first round and c...,"June 23, 2020",Chris Santos,Scott Conant,Erik Ramirez,"rabbit terrine, guanciale, spring garlic, bur...",...,Jose Luis Chavez,Chef and Owner from New York NY,Matt Greiner,Executive Chef from Raleigh NC,Mimi Weissenborn,Executive Chef from Harlem NY,Nemo Bolin,Chef and Owner from Providence RI,Nemo Bolin,True
565,45,10,564,"""Time and Turmoil""",Chef Arden forgot an ingredient in the first r...,"June 30, 2020",Amanda Freitag,Maneet Chauhan,Scott Conant,"hash brown patties, Manila clams, escarole, b...",...,Lindsay Smith-Rosales,Chef and Owner from Laguna Beach CA,Arden Lewis,Executive Chef from New York NY,Lina Zarcaro,Private Chef from Bradley Beach NJ,Luca Annunziata,Executive Chef from Charlotte NC,Luca Annunziata,False
566,45,11,565,"""Jarring Jars""",The guest judge in this episode was Chef Ray G...,"July 7, 2020",Scott Conant,Geoffrey Zakarian,Ray Garcia,"sea beans, dehydrated carrot sticks, egg coff...",...,May Siricharoen,Executive Chef from Los Angeles CA,Chris Day,Executive Sous Chef from Boston MA,Patrick McKee,Executive Chef from Portland OR,Phillip Esteban,Research & Development Chef from San Diego CA,Phillip Esteban,True
567,45,12,566,"""Cauliflower Power""",In this unofficially vegetarian themed episode...,"July 21, 2020",Maneet Chauhan,Marc Murphy,Esther Choi,"cauliflower avocado toast, cauliflower rice, ...",...,Manjit Manohar,Executive Sous Chef from N

In [7]:
# Count of injuries reported (cuts and burns)
chopped_df.injured.value_counts()

False    421
True      35
Name: injured, dtype: int64

In [8]:
# functions used on the textual columns entries to alter Nan spaces
def renameNan(val):
    if val == "True":
        return(val)
    elif val == "False":
        return(val)
    else:
        return(val.replace('nan', 'False'))
    
def obtain_FName(val):
    return(val.split(" ")[1])

def blankEpisodeNotes(val):
    if val != "nan":
        return val
    else:
        return(val.replace('nan', ' '))


In [9]:
# applying functions to string columns
chopped_df['injured'] = chopped_df['injured'].astype('str')
chopped_df['episode_notes'] = chopped_df['episode_notes'].astype('str')
chopped_df['injured'] = chopped_df['injured'].apply(renameNan)

In [10]:
chopped_df.head()

,season,season_episode,series_episode,episode_name,episode_notes,air_date,judge1,judge2,judge3,appetizer,...,contestant1,contestant1_info,contestant2,contestant2_info,contestant3,contestant3_info,contestant4,contestant4_info,winner,injured
0,1,1,1,"""Octopus, Duck, Animal Crackers""",This is the first episode with only three offi...,"January 13, 2009",Marc Murphy,Alex Guarnaschelli,Aarón Sánchez,"baby octopus, bok choy, oyster sauce, smoked ...",...,Summer Kriegshauser,Private Chef and Nutrition Coach New York NY,Perry Pollaci,Private Chef and Sous chef Bar Blanc New Yo...,Katie Rosenhouse,Pastry Chef Olana Restaurant New York NY,Sandy Davis,Catering Chef Showstoppers Catering at Union...,Sandy Davis,False
1,1,2,2,"""Tofu, Blueberries, Oysters""",This is the first of a few episodes with five ...,"January 20, 2009",Aarón Sánchez,Alex Guarnaschelli,Marc Murphy,"firm tofu, tomato paste, prosciutto",...,Raymond Jackson,Private Caterer and Culinary Instructor West...,Klaus Kronsteiner,Chef de cuisine Liberty National Golf Course...,Christopher Jackson,Executive Chef and Owner Ted and Honey Broo...,Pippa Calland,Owner and Chef Chef for Hire LLC Newville PA,Pippa Calland,False
2,1,3,3,"""Avocado, Tahini, Bran Flakes""",nan,"January 27, 2009",Aarón Sánchez,Alex Guarnaschelli,Marc Murphy,"lump crab meat, dried shiitake mushrooms, pin...",...,Margaritte Malfy,Executive Chef and Co-owner La Palapa New Y...,Rachelle Rodwell,Chef de cuisine SoHo Grand Hotel New York NY,Chris Burke,Private Chef New York NY,Andre Marrero,Chef tournant L’Atelier de Joël Robuchon Ne...,Andre Marrero,False
3,1,4,4,"""Banana, Collard Greens, Grits""","In the appetizer round, Chef Chuboda refused t...","February 3, 2009",Scott Conant,Amanda Freitag,Geoffrey Zakarian,"ground beef, wonton wrappers, cream of mushro...",...,Sean Chudoba,Executive Chef Ayza Wine Bar New York NY,Kyle Shadix,Chef Registered Dietician and Culinary Consu...,Luis Gonzales,Executive Chef Knickerbocker Bar & Grill Ne...,Einat Admony,Chef and Owner Taïm New York NY,Einat Admony,False
4,1,5,5,"""Yucca, Watermelon, Tortillas""",nan,"February 10, 2009",Geoffrey Zakarian,Alex Guarnaschelli,Marc Murphy,"watermelon, canned sardines, pepper jack chee...",...,John Keller,Personal Chef New York NY,Andrea Bergquist,Executive Chef New York NY,Ed Witt,Executive Chef / Wine Director Bloomingdale ...,Josh Emett,Chef de cuisine Gordon Ramsay at The London ...,Josh Emett,False


In [11]:
# Obtaining the first name of winning contestant to search if injured chef
chopped_df['winner_Fname'] = chopped_df['winner'].apply(obtain_FName)

In [12]:
chopped_df.head()

,season,season_episode,series_episode,episode_name,episode_notes,air_date,judge1,judge2,judge3,appetizer,...,contestant1_info,contestant2,contestant2_info,contestant3,contestant3_info,contestant4,contestant4_info,winner,injured,winner_Fname
0,1,1,1,"""Octopus, Duck, Animal Crackers""",This is the first episode with only three offi...,"January 13, 2009",Marc Murphy,Alex Guarnaschelli,Aarón Sánchez,"baby octopus, bok choy, oyster sauce, smoked ...",...,Private Chef and Nutrition Coach New York NY,Perry Pollaci,Private Chef and Sous chef Bar Blanc New Yo...,Katie Rosenhouse,Pastry Chef Olana Restaurant New York NY,Sandy Davis,Catering Chef Showstoppers Catering at Union...,Sandy Davis,False,Sandy
1,1,2,2,"""Tofu, Blueberries, Oysters""",This is the first of a few episodes with five ...,"January 20, 2009",Aarón Sánchez,Alex Guarnaschelli,Marc Murphy,"firm tofu, tomato paste, prosciutto",...,Private Caterer and Culinary Instructor West...,Klaus Kronsteiner,Chef de cuisine Liberty National Golf Course...,Christopher Jackson,Executive Chef and Owner Ted and Honey Broo...,Pippa Calland,Owner and Chef Chef for Hire LLC Newville PA,Pippa Calland,False,Pippa
2,1,3,3,"""Avocado, Tahini, Bran Flakes""",nan,"January 27, 2009",Aarón Sánchez,Alex Guarnaschelli,Marc Murphy,"lump crab meat, dried shiitake mushrooms, pin...",...,Executive Chef and Co-owner La Palapa New Y...,Rachelle Rodwell,Chef de cuisine SoHo Grand Hotel New York NY,Chris Burke,Private Chef New York NY,Andre Marrero,Chef tournant L’Atelier de Joël Robuchon Ne...,Andre Marrero,False,Andre
3,1,4,4,"""Banana, Collard Greens, Grits""","In the appetizer round, Chef Chuboda refused t...","February 3, 2009",Scott Conant,Amanda Freitag,Geoffrey Zakarian,"ground beef, wonton wrappers, cream of mushro...",...,Executive Chef Ayza Wine Bar New York NY,Kyle Shadix,Chef Registered Dietician and Culinary Consu...,Luis Gonzales,Executive Chef Knickerbocker Bar & Grill Ne...,Einat Admony,Chef and Owner Taïm New York NY,Einat Admony,False,Einat
4,1,5,5,"""Yucca, Watermelon, Tortillas""",nan,"February 10, 2009",Geoffrey Zakarian,Alex Guarnaschelli,Marc Murphy,"watermelon, canned sardines, pepper jack chee...",...,Personal Chef New York NY,Andrea Bergquist,Executive Chef New York NY,Ed Witt,Executive Chef / Wine Director Bloomingdale ...,Josh Emett,Chef de cuisine Gordon Ramsay at The London ...,Josh Emett,False,Josh


In [13]:
# Determining if winning contestant was injured during the competition
chopped_df['injured_winner'] = chopped_df.apply(lambda x: x.winner_Fname in x.episode_notes, axis=1)

In [14]:
chopped_df.head()

,season,season_episode,series_episode,episode_name,episode_notes,air_date,judge1,judge2,judge3,appetizer,...,contestant2,contestant2_info,contestant3,contestant3_info,contestant4,contestant4_info,winner,injured,winner_Fname,injured_winner
0,1,1,1,"""Octopus, Duck, Animal Crackers""",This is the first episode with only three offi...,"January 13, 2009",Marc Murphy,Alex Guarnaschelli,Aarón Sánchez,"baby octopus, bok choy, oyster sauce, smoked ...",...,Perry Pollaci,Private Chef and Sous chef Bar Blanc New Yo...,Katie Rosenhouse,Pastry Chef Olana Restaurant New York NY,Sandy Davis,Catering Chef Showstoppers Catering at Union...,Sandy Davis,False,Sandy,False
1,1,2,2,"""Tofu, Blueberries, Oysters""",This is the first of a few episodes with five ...,"January 20, 2009",Aarón Sánchez,Alex Guarnaschelli,Marc Murphy,"firm tofu, tomato paste, prosciutto",...,Klaus Kronsteiner,Chef de cuisine Liberty National Golf Course...,Christopher Jackson,Executive Chef and Owner Ted and Honey Broo...,Pippa Calland,Owner and Chef Chef for Hire LLC Newville PA,Pippa Calland,False,Pippa,False
2,1,3,3,"""Avocado, Tahini, Bran Flakes""",nan,"January 27, 2009",Aarón Sánchez,Alex Guarnaschelli,Marc Murphy,"lump crab meat, dried shiitake mushrooms, pin...",...,Rachelle Rodwell,Chef de cuisine SoHo Grand Hotel New York NY,Chris Burke,Private Chef New York NY,Andre Marrero,Chef tournant L’Atelier de Joël Robuchon Ne...,Andre Marrero,False,Andre,False
3,1,4,4,"""Banana, Collard Greens, Grits""","In the appetizer round, Chef Chuboda refused t...","February 3, 2009",Scott Conant,Amanda Freitag,Geoffrey Zakarian,"ground beef, wonton wrappers, cream of mushro...",...,Kyle Shadix,Chef Registered Dietician and Culinary Consu...,Luis Gonzales,Executive Chef Knickerbocker Bar & Grill Ne...,Einat Admony,Chef and Owner Taïm New York NY,Einat Admony,False,Einat,False
4,1,5,5,"""Yucca, Watermelon, Tortillas""",nan,"February 10, 2009",Geoffrey Zakarian,Alex Guarnaschelli,Marc Murphy,"watermelon, canned sardines, pepper jack chee...",...,Andrea Bergquist,Executive Chef New York NY,Ed Witt,Executive Chef / Wine Director Bloomingdale ...,Josh Emett,Chef de cuisine Gordon Ramsay at The London ...,Josh Emett,False,Josh,False


In [15]:
# Subset of data containing epsiodes where a contestant was injured
chopped_df_injured = chopped_df[chopped_df['injured']=='True']
chopped_df_injured

,season,season_episode,series_episode,episode_name,episode_notes,air_date,judge1,judge2,judge3,appetizer,...,contestant2,contestant2_info,contestant3,contestant3_info,contestant4,contestant4_info,winner,injured,winner_Fname,injured_winner
21,2,9,22,"""Buckwheat Blunders and Twists of Fate""",Chef Amitzur cut himself in Round 1 and contin...,"August 25, 2009",Chris Santos,Alex Guarnaschelli,Aarón Sánchez,"olive oil packed tuna, English cucumbers, buc...",...,Rory O'Farrell,Executive Chef Ayza Wine & Chocolate Bar Ne...,Mor Amitzur,Executive Chef The View at the Marriott Marq...,Diane DiMeo,Private Chef Red Box Bistro New York NY,Diane DiMeo,True,Diane,False
27,3,2,28,"""Oh My Goshy, Umeboshi""",Chef Esmonde cut herself during the entrée rou...,"October 20, 2009",Aarón Sánchez,Jody Williams,Geoffrey Zakarian,"peas, peaches, eel",...,Barbara Esmonde,Executive Chef Mediterranean Catering Wynne...,Peter Karapanagiotis,Executive Chef Privé Mediterranean Restauran...,Jeremy Duclut,Executive Chef Georges' Bar and Restaurant ...,Jeremy Duclut,True,Jeremy,False
52,5,1,53,"""""Dream'n of Redeem'n""""",This episode featured four past runners-up who...,"July 20, 2010",Geoffrey Zakarian,Alex Guarnaschelli,Marc Murphy,"rump roast, rainbow chard, vegetable yeast sp...",...,Lola Garand,Freelance Consulting Chef New York NY Epis...,Dominick Feragola,Sous chef Broadway East New York NY Episo...,Matthew Zappoli,Owner and Executive Chef Tre Amici Long Bra...,Matthew Zappoli,True,Matthew,False
68,6,4,69,"""Doughs and Don'ts""",Chef Santorelli cut herself badly in the appet...,"January 25, 2011",Scott Conant,Amanda Freitag,Chris Santo,"saba, garlic chutes, Canadian bacon, trout",...,Monica Byrne,Chef and Restaurateur Home/Made Brooklyn NY,Anna Maria Santorelli,Chef and Restaurateur Anna Maria's Larchmon...,James Gillespie,Culinary Instructor New York NY,James Gillespie,True,James,False
77,7,1,78,"""The Icing on the Steak""",Chef Fox cut herself in Round 1.,"May 3, 2011",Chris Santos,Amanda Freitag,Marc Murphy,"smoked turkey legs, ramen noodles, cherry bom...",...,Marco Suarez,Executive Chef Ledge Kitchen and Drinks Dor...,Tryg Siverson,Chef and Restaurateur Friedman's Lunch New ...,Shehu Fitzgerald,Garde manger Chef Marriott New York Marquis ...,Shehu Fitzgerald,True,Shehu,False
85,7,9,86,"""Thyme Flies""",In the entree Chef Claude-Mendy forgot an ingr...,"June 28, 2011",Aarón Sánchez,Alex Guarnaschelli,Marc Murphy,"Korean-style short ribs, lemon thyme, zucchin...",...,James Lee,Chef and Restaurateur New York NY,April Stamm,Chef Team Leader Prepared foods at Whole Foo...,Marie-Claude Mendy,Executive Chef/Owner Teranga Boston MA,Marie-Claude Mendy,True,Marie-Claude,False
89,8,3,90,"""Sweet Second Chance!""",This episode featured four past non-winners. T...,"July 26, 2011",Geoffrey Zakarian,Amanda Freitag,Marc Murphy,"pancake mix, guava nectar, Aleppo peppers, As...",...,Siggy Sollitto,Chef and Restaurateur Siggy's Good Food Bro...,Yoanne Magris,Chef and Restaurateur Yo In Yo Out New York...,Lance Nitahara,Executive Chef Camp-of-the-Woods Speculator...,Lance Nitahara,True,Lance,False
147,12,13,148,"""For Sake's Sake""",Chef Boudreaux cut himself at the end of the a...,"November 20, 2012",Chris Santos,Amanda Freitag,Marc Murphy,"ostrich tenderloin, Anzac biscuits, cauliflow...",...,Tasheena Butler,Chef and Caterer T. Marie's Kitchen & Cateri...,Derek Gigliotti,Executive Chef Onda Ristorante Mirage Resor...,Ria Pell,Chef and Restaurateur Ria's Bluebird / Sauce...,Ria Pell,True,Ria,False
178,15,6,179,"""Amazing Amateurs""",This was an official amateurs episode which wa...,"May 14, 2013",Geoffrey Zakarian,Alex Guarnaschelli,Aarón Sánchez,"dragon fruit, guanciale, sunflower sprouts, s...",...,Melissa Campbell,Training Administrator from Los Angeles CA,Rique Uresti,Spin Instructor from New York NY,Marisa Biaggi,Content Producer and Strategist/PR from New Y...,Marisa Biaggi,True,Marisa,True
185,15,13,186,"""Wurst Case Scenario""",Chef Harriott cut herself in

In [16]:
# Subset of data containing episodes of contestant who were injured and won
chopped_df_injured_win = chopped_df_injured[chopped_df_injured['injured_winner']==True]
chopped_df_injured_win

,season,season_episode,series_episode,episode_name,episode_notes,air_date,judge1,judge2,judge3,appetizer,...,contestant2,contestant2_info,contestant3,contestant3_info,contestant4,contestant4_info,winner,injured,winner_Fname,injured_winner
178,15,6,179,"""Amazing Amateurs""",This was an official amateurs episode which wa...,"May 14, 2013",Geoffrey Zakarian,Alex Guarnaschelli,Aarón Sánchez,"dragon fruit, guanciale, sunflower sprouts, s...",...,Melissa Campbell,Training Administrator from Los Angeles CA,Rique Uresti,Spin Instructor from New York NY,Marisa Biaggi,Content Producer and Strategist/PR from New Y...,Marisa Biaggi,True,Marisa,True
209,17,11,210,"""Redeemed or Re-chopped?""",This episode featured four past runners-up. In...,"November 5, 2013",Marc Murphy,Alex Guarnaschelli,Aarón Sánchez,"vegan lobster, hot mustard, canned chop suey,...",...,Jeff Eng,Executive Chef Clyde's Tower Oaks Lodge Roc...,Kent Rollins,"Episode 12.17 – ""Grill Masters: Part Four"" and...",Zoe Feigenbaum,Chef and Restaurateur Zoë New York NY Epi...,Zoe Feigenbaum,True,Zoe,True
490,39,8,491,"""You've Got Snail""",The stuffed croissant in the second round was ...,"April 9, 2019",Scott Conant,Amanda Freitag,Maneet Chauhan,"whelks, carbonated candy, Fresno chiles, baby...",...,Darius Chase,Sous Chef from Louisville KY,Aless Mac Carthy,Owner/Caterer from New York NY,Anna Vocaturo,Chef Partner from Washington D.C,Anna Vocaturo,True,Anna,True
523,42,2,524,"""Weird, Wacky, and Wild!""",The theme of this episode was weird ingredient...,"September 17, 2019",Geoffrey Zakarian,Amanda Freitag,Marcus Samuelsson,"foot soup, buffalo milk pasta, bitter melon l...",...,Greg Stott,Chef de Cuisine from Ramsey NJ,Alex Crabb,Chef and Owner from Boston MA,Jen Biesty,Chef and Owner from Oakland CA,Jen Biesty,True,Jen,True
529,42,8,528,"""Poke Power""","Chef Ethan cut himself in the first round, for...","June 2, 2020",Amanda Freitag,Tiffani Faison,Scott Conant,"poke pizza, tomatillos, cuttlefish, roasted r...",...,Phil DeMaiolo,Executive Chef from New York NY,Cristina Espinosa,Chef from Belmont CA,Ethan Speizer,Estate Chef from Napa CA,Ethan Speizer,True,Ethan,True
547,44,5,546,"""Meat Fight""",This episode featured beef in every basket. Th...,"November 26, 2019",Scott Conant,Chris Santos,Angie Mar,"dino beef ribs, cherry coconut meat bars, ice...",...,Tomer Avital,Executive Chef from New York NY,"James ""Mac"" Moran",Executive Chef from New York NY,Chris Coleman,Executive Chef from Charlotte NC,Chris Coleman,True,Chris,True
553,44,11,552,"""A Pretty Pickle Pizza""","In round 2, Chef Saransh only got duck on one ...","March 10, 2020",Chris Santos,Maneet Chauhan,Geoffrey Zakarian,"tuna loin, asparagus, red quinoa, pickle pizza",...,Saransh Oberoi,Chef and Owner from San Diego CA,Xzherieh Norris,Sous Chef from Brooklyn NY,Javier Canteras,Chef and Owner from Portland OR,Javier Canteras,True,Javier,True


In [17]:
#converting True to 1 and failed to 0
chopped_df['injured'] = (chopped_df['injured'] == True).astype(int)
chopped_df['injured_winner'] = (chopped_df['injured_winner'] == True).astype(int)

In [18]:
# subset of data for modeling
dropped_cols = ['season', 'season_episode', 'series_episode', 'episode_notes', 'episode_name', 'air_date']
chopped_df_model = chopped_df.drop(dropped_cols, axis=1)

In [19]:
# Subset of data without the food
dropped_cols_food = ['season', 'season_episode', 'series_episode', 'episode_notes', 'episode_name', 'air_date', 'appetizer', 'entree', 'dessert', 'contestant1_info', 'contestant2_info', 'contestant3_info', 'contestant4_info', 'winner_Fname']
chopped_df_foodless = chopped_df.drop(dropped_cols_food, axis=1)

In [20]:
for c in chopped_df_foodless.columns:
    col_type = chopped_df_foodless[c].dtype
    if col_type == 'object' :
        a=chopped_df_foodless[c].unique()
        keys= range(a.shape[0])
        diction={}
        for idx,val in enumerate(a):
            diction[idx] = a[idx]
        diction = {v: k for k, v in diction.items()}
        print(diction)
        #kick_projects_copy[c] = [diction[item] for item in kick_projects_copy[c]] 
        #kick_projects_copy[c] = kick_projects_copy[c].astype('category')

{'Marc Murphy': 0, 'Aarón Sánchez': 1, 'Scott Conant': 2, 'Geoffrey Zakarian': 3, 'Chris Santos': 4, 'Josh Capon': 5, 'Mark Bittman': 6, 'Marcus Samuelsson': 7, 'Amanda Freitag': 8, 'Zakary Pelaccio': 9, 'Seamus Mullen': 10, 'Maneet Chauhan': 11, 'Elizabeth Karmel': 12, 'Joseph Brown': 13, 'Alex Guarnaschelli': 14, 'Edi Frauneder': 15, 'Peter Oleyer': 16, 'Aarón Sanchez': 17, 'Michael Chernow': 18, 'Andrew Zimmern': 19, 'Christina Tosi': 20, 'Michelle Bernstein': 21, 'Natasha Case': 22, 'Jeff Mauro': 23, 'Adam Moskowitz': 24, 'Christian Petroni': 25, 'Anne Burrell': 26, 'Bobby Flay': 27, 'Nancy Silverton': 28, 'Alton Brown': 29, 'Adam Sobel': 30, 'Martha Stewart': 31, "Stuart O'Keeffe": 32, 'Angie Mar': 33, 'Geoffrey Zacharian': 34, 'Maureen McCormick': 35, 'Barry Williams': 36, nan: 37}
{'Alex Guarnaschelli': 0, 'Amanda Freitag': 1, 'Jody Williams': 2, 'Geoffrey Zakarian': 3, 'Scott Conant': 4, 'Sue Torres': 5, 'Maneet Chauhan': 6, 'Missy Robbins': 7, 'Chris Santos': 8, 'Elizabeth Kar

In [21]:
# Get dummy data for model
chopped_df_foodless_ip = pd.get_dummies(chopped_df_foodless)

In [22]:
# Create feature list for independent and dependent variables
features=list(chopped_df_foodless_ip)
features.remove('injured_winner')
response= ['injured_winner']

In [23]:
# Normalize data for modeling
chopped_df_foodless_ip_scaled_ftrs = pd.DataFrame(preprocessing.normalize(chopped_df_foodless_ip[features]))
chopped_df_foodless_ip_scaled_ftrs.columns=list(chopped_df_foodless_ip[features])

In [24]:
chopped_df_foodless_ip_scaled_ftrs[:3]

,injured,judge1_Aarón Sanchez,judge1_Aarón Sánchez,judge1_Adam Moskowitz,judge1_Adam Sobel,judge1_Alex Guarnaschelli,judge1_Alton Brown,judge1_Amanda Freitag,judge1_Andrew Zimmern,judge1_Angie Mar,...,winner_ Wade Burch,winner_ Walter D'Rozario,winner_ Wayne Lyons,winner_ Winnette McIntosh Ambrose,winner_ Xavier Santiago,winner_ Yazmene Kaylani,winner_ Ype Von Hengst,winner_ Yuri Szarzewski,winner_ Zeb Stevenson,winner_ Zoe Feigenbaum
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.353553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.353553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
#train the mdodel
X_train, X_test, y_train, y_test = train_test_split(chopped_df_foodless_ip_scaled_ftrs, chopped_df_foodless_ip[response], test_size=0.30)

In [26]:
# Create and fir Decision Tree Model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [27]:
# Predicition of model
y_pred = model.predict(X_test)

In [28]:
# Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       164
           1       0.00      0.00      0.00         7

    accuracy                           0.96       171
   macro avg       0.48      0.50      0.49       171
weighted avg       0.92      0.96      0.94       171



C:\Users\Gabe\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
# Accuracy of Model
accuracy = np.mean(cross_val_score(model, X_test, y_test, scoring='accuracy')) * 100
print("Accuracy: {}%".format(accuracy))

Accuracy: 95.91579523521449%


C:\Users\Gabe\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [30]:
# Confusion Matrix
print('confusion matrix {}'.format(pd.DataFrame(
            confusion_matrix(y_test, y_pred),
            columns=['Predicted Loss', 'Predicted Win'],
            index=['True Loss', 'True Win']
        )))

confusion matrix            Predicted Loss  Predicted Win
True Loss             164              0
True Win                7              0


In [31]:
# Predict the on the train_data
X_test["Pred_win"] = model.predict(X_test[features])

# Predict the on the train_data
X_train["Pred_win"] = model.predict(X_train[features])

# Predict the on the train_data
chopped_df_foodless_ip["Pred_win"] = model.predict(chopped_df_foodless_ip_scaled_ftrs)

C:\Users\Gabe\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Gabe\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
